In [ ]:
%run "../../FUNCOES/Classes_1.1"

In [ ]:
###### determina local de leitura 
delimiter    = "|"
schema_name  = "mawe_bronze"
source_table = "ADR6_TEMPLATE"
source_path_landing = f"dbfs:/mnt/0-transientde/SAP/Coletado/{source_table}"
#Years = ['2018','2019','2020','2021','2022']  
#Years = ['2013','2014','2015','2016','2017']  
#Years = ['2008','2009','2010','2011','2012']  
#Years = ['2003','2004','2005','2006','2007']  
#Years = ['1999','2000','2001','2002'] 

#### monta lista de arquivos a serem lidos
landing_location = Framework.folder_read_path(source_table, source_path_landing)

### Caminho de leitura deveria acompanhar os meses tbm
df = Framework.read_landing_zone_dataframe(
    file_format=Framework.RawFileFormat.CSV,
    location=landing_location,
    delimiter=delimiter
)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1561939818493312> in <cell line: 16>()
     14 
     15 ### Caminho de leitura deveria acompanhar os meses tbm
---> 16 df = Framework.read_landing_zone_dataframe(
     17     file_format=Framework.RawFileFormat.CSV,
     18     location=landing_location,

<command-3136467356617305> in read_landing_zone_dataframe(cls, file_format, location, delimiter, quote)
    132         ###try:
    133         df = (
--> 134             spark.read
    135             .format(file_format.lower())
    136             .option("header", True)

/databricks/spark/python/pyspark/sql/readwriter.py in load(self, path, format, schema, **options)
    180                 path = [path]  # type: ignore[list-item]
    181             assert self._spark._sc._jvm is not None
--> 182             return self._df(self._jreader.load(self._spark._sc._jvm.PythonUt

In [ ]:
#### gera data de extração em tabela que nao possui na origem
# end   = date.today()
# end_formated   = end.strftime("%Y%m%d")
# #df = df.withColumn("DATA_EXTRACAO", lit(end_formated))
# df = df.withColumn("ANO_EXTRACAO", F.substring(lit(end_formated),1,4))
# df = df.withColumn("MES_EXTRACAO", F.substring(lit(end_formated),1,6))
# df = df.withColumn("MES_FIXACAO_PRECO", F.substring(df.KDATU,1,6) )

df = df.withColumn("ANO_EXTRACAO", F.substring(df.DATA_EXTRACAO,1,4))
df = df.withColumn("MES_EXTRACAO", F.substring(df.DATA_EXTRACAO,1,6))
# df = df.withColumn("MES_CRIACAO", F.substring(df.ERDAT,1,6))



#### em caso de criação de colunas business ou processing data na delta table e aplica MD5
bk_columns_keys= ['ADDRNUMBER',
'CONSNUMBER',
'SMTP_ADDR',
'SMTP_SRCH']


df = df.withColumn("BK_EXTRACAO", F.md5(F.concat_ws('_', array(bk_columns_keys))))

In [ ]:
###### Monta variaveis da Delta Table
target_table = "ADR6"
key_columns  = ["BK_EXTRACAO"]
partition    = "DATA_EXTRACAO"

###### busca caminho delta table na camada
location = Framework.generate_bronze_table_location(
    table_name=target_table,
)

## executa a gravação da tabela bronze
results = Framework.write_delta_table(
    df=df,
    location=location,
    schema_name=schema_name,
    table_name=target_table,
    load_type=Framework.LoadType.APPEND_ALL,
    key_columns=key_columns,
    partition_columns=partition,
    schema_evolution_mode=Framework.SchemaEvolutionMode.ADD_NEW_COLUMNS,
)

In [ ]:
# movimenta os arquivos entre as pastas 
movimenta_arquivos = Framework.transport_collected_precessed(landing_location)